In [ ]:
rush_hour = "晨峰"    #晨峰或昏峰

import pandas as pd

df= pd.read_excel("TRTS-4S屏柵線.xlsx", sheet_name=None)
sheet1 = df["節點編號"]


ue_results_1 = pd.read_csv(f"104_現況_{rush_hour}/104_現況_{rush_hour}_UE_results.dat", sep="\t")
ue_results_2 = pd.read_csv(f"104_全線拆除_{rush_hour}/104_全線拆除_{rush_hour}_UE_results.dat", sep="\t")
ue_results_3 = pd.read_csv(f"104_部分拆除_{rush_hour}/104_部分拆除_{rush_hour}_UE_results.dat", sep="\t")

ue_results_1["link_name"] = ue_results_1.apply(lambda row: f"{int(row['tailNode'])}_{int(row['headNode'])}", axis=1)
ue_results_2["link_name"] = ue_results_2.apply(lambda row: f"{int(row['tailNode'])}_{int(row['headNode'])}", axis=1)
ue_results_3["link_name"] = ue_results_3.apply(lambda row: f"{int(row['tailNode'])}_{int(row['headNode'])}", axis=1)

ue_results_1 = ue_results_1.set_index("link_name")
ue_results_2 = ue_results_2.set_index("link_name")
ue_results_3 = ue_results_3.set_index("link_name")

sheet1["link_name1"] = sheet1.apply(lambda row: f"{int(row['A1'])}_{int(row['B1'])}", axis=1)
sheet1["link_name2"] = sheet1.apply(lambda row: f"{int(row['A2'])}_{int(row['B2'])}", axis=1)

#將承德橋機車道流量合併
ue_results_1.loc['10246_6631', "UE_flow"] += ue_results_1.loc['10245_6631', "UE_flow"]
ue_results_1.loc['6631_10247', "UE_flow"] += ue_results_1.loc['6631_10248', "UE_flow"]
ue_results_2.loc['10246_6631', "UE_flow"] += ue_results_1.loc['10245_6631', "UE_flow"]
ue_results_2.loc['6631_10247', "UE_flow"] += ue_results_1.loc['6631_10248', "UE_flow"]
ue_results_3.loc['10246_6631', "UE_flow"] += ue_results_1.loc['10245_6631', "UE_flow"]
ue_results_3.loc['6631_10247', "UE_flow"] += ue_results_1.loc['6631_10248', "UE_flow"]
#將中山北路機車道流量合併
ue_results_1.loc['6448_6420', "UE_flow"] += ue_results_1.loc['8371_8376', "UE_flow"]
ue_results_1.loc['8375_8372', "UE_flow"] += ue_results_1.loc['8374_8373', "UE_flow"]
ue_results_2.loc['6448_6420', "UE_flow"] += ue_results_1.loc['8371_8376', "UE_flow"]
ue_results_2.loc['8375_8372', "UE_flow"] += ue_results_1.loc['8374_8373', "UE_flow"]
ue_results_3.loc['6448_6420', "UE_flow"] += ue_results_1.loc['8371_8376', "UE_flow"]
ue_results_3.loc['8375_8372', "UE_flow"] += ue_results_1.loc['8374_8373', "UE_flow"]

for index, row in sheet1.iterrows():
    group = row['group']
    if group == 'SL1' and row['link'] != '承德橋(機車道)':
        link_name1 = row['link_name1']
        link_name2 = row['link_name2']
        print(f'''{row['link']}
        【南向】  全線拆除: {(ue_results_2.loc[link_name1, "UE_flow"]-ue_results_1.loc[link_name1, "UE_flow"])/ue_results_1.loc[link_name1, "UE_flow"]* 100:.2f}%，部分拆除: {(ue_results_3.loc[link_name1, "UE_flow"]-ue_results_1.loc[link_name1, "UE_flow"])/ue_results_1.loc[link_name1, "UE_flow"]* 100:.2f}%
        【北向】  全線拆除: {(ue_results_2.loc[link_name2, "UE_flow"]-ue_results_1.loc[link_name2, "UE_flow"])/ue_results_1.loc[link_name2, "UE_flow"]* 100:.2f}%，部分拆除: {(ue_results_3.loc[link_name2, "UE_flow"]-ue_results_1.loc[link_name2, "UE_flow"])/ue_results_1.loc[link_name2, "UE_flow"]* 100:.2f}%
        ''')



#{link_name1}, {link_name2}